TPGshipのシミュレーション動画作成プログラム

ライブラリ読み込み

In [1]:
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import polars as pl
import numpy as np
import math
from numpy import linalg as LA
import cartopy.feature as cfeature
from PIL import Image
import matplotlib.patches as patches
from datetime import datetime, timedelta, timezone
from dateutil import tz


データファイル読み取り

台風＆TPGship経路データ

In [2]:
year = 2018

ship_typhoon_route_data = pl.read_csv(str(year) + "_sim_ver4.csv",encoding="shift-jis")

typhoon_data = pl.read_csv("typhoon_data_6hour_intervals_verpl/table" + str(year) + "_6_interval.csv",encoding="shift-jis")

TPGshipの動作画像化

In [ ]:
UTC = timezone(timedelta(hours =+ 0),"UTC")

for j in range(len(ship_typhoon_route_data)):
#for j in range(3):
    #j_ori = j
    #j = j + 342
    #地図の作成
    fig = plt.figure(figsize=(12,16)) # プロット領域の作成（matplotlib）
    ax = fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree()) # サブプロット作成時にcartopy呼び出し

    ax.set_facecolor('paleturquoise')
    land_h = cfeature.NaturalEarthFeature('physical', 'land', '50m')
    ax.add_feature(land_h,color='g')
    ax.set_extent([120, 180, 0, 70], ccrs.Geodetic())

    

    #拠点&待機位置
    base_lat = 24
    base_lon = 153
    ax.plot(base_lon,base_lat,'crimson',markersize=15,marker="d")


    #陸地境界の設定
    #nonaggression_line_lat = [0,13,13,15,15,24,24,26,26,28,28,32.2,32.2,34,34,41.2,41.2,44,44,50,50]
    #nonaggression_line_lon = [127.5,127.5,125,125,123,123,126,126,130.1,130.1,132.4,132.4,137.2,137.2,143,143,149,149,156,156,180]
    #for i in range(len(nonaggression_line_lat)-1):
    #    ax.plot([nonaggression_line_lon[i],nonaggression_line_lon[i+1]],[nonaggression_line_lat[i],nonaggression_line_lat[i+1]],'red',linewidth=4)

    #台風の作成 
    unixtime = ship_typhoon_route_data[j,"unixtime"]
    
    TYdata = typhoon_data.filter((pl.col("unixtime") == unixtime))
    typhoon_lon = TYdata[:]["LON"]
    typhoon_lat = TYdata[:]["LAT"]
    if len(typhoon_lon) != 0:
        for k in range(len(typhoon_lon)):
            ax.plot(typhoon_lon[k],typhoon_lat[k],'grey',markersize=50,marker="o",alpha=0.8)


    #凡例の作成
    view_lon = 170
    text_lon = 173
    d_view_lat = 3
    view_ship_lon = 150
    view_ship_lat = 60
    view_lat_1 = 58
    lon_width = 2

    ship_text_lon = 154
    ship_text_lat = 63.5

    #目標名の設定
    target = ship_typhoon_route_data[j,"TARGET LOCATION"]
    if ship_typhoon_route_data[j,"TARGET TYPHOON"] != 0:
        target = "Typhoon " + str(target)
    else:
        target = str(target)
    
    #船速取得
    speed = str(format(ship_typhoon_route_data[j,"SHIP SPEED[kt]"], '.1f'))

    #状態表示　移動・発電・待機
    if (ship_typhoon_route_data[j,"SHIP SPEED[kt]"] == 0)&(ship_typhoon_route_data[j,"TIMESTEP POWER GENERATION[Wh]"] == 0) & (ship_typhoon_route_data[j,"TIMESTEP POWER CONSUMPTION[Wh]"] == 0):
        state = "Standby"
        iro = "lime"
    elif ship_typhoon_route_data[j,"TIMESTEP POWER GENERATION[Wh]"] > 0:
        state = "Power Generation"
        speed = speed + "(prov)"
        iro = "yellow"
    elif (ship_typhoon_route_data[j,"TIMESTEP POWER CONSUMPTION[Wh]"] > 0) or (ship_typhoon_route_data[j,"SHIP SPEED[kt]"] > 0):
        state = "Moving"
        iro = "red"
    
    else:
        print("Error")

    #船内蓄電量
    storage_per = str(format(ship_typhoon_route_data[j,"ONBOARD POWER STORAGE PER[%]"], '.1f'))

    #TPGship関連情報表示
    r = patches.Rectangle(xy=(145, 58), width=40, height=13, ec='k', fc='w',zorder=3)
    ax.add_patch(r)
    

    ax.quiver(view_ship_lon, view_ship_lat, 0, 55, color=iro, edgecolor="k", linewidth = 1.0, headwidth=30, headlength=50 , headaxislength=50, width=.02, scale=400.,zorder=4)
    ax.text(ship_text_lon, ship_text_lat+5.2, "TPGship prototype", size=25, color="black",zorder=4)
    ax.text(ship_text_lon, ship_text_lat+2.5, "Target          : "+target, size=18, color="black",zorder=4)
    ax.text(ship_text_lon, ship_text_lat-0.0, "States          : "+state, size=18, color="black",zorder=4)
    ax.text(ship_text_lon, ship_text_lat-2.5, "Speed[kt]    : "+speed, size=18, color="black",zorder=4)
    ax.text(ship_text_lon, ship_text_lat-5.0, "Storage[%]  : "+storage_per, size=18, color="black",zorder=4)


    #日数記録
    date = str(datetime.fromtimestamp(unixtime,UTC))
    r = patches.Rectangle(xy=(145, 54), width=24, height=4, ec='k', fc='w',zorder=3)
    ax.add_patch(r)
    ax.text(145.5, view_lat_1 - 0.8*d_view_lat-0.4, date, size=16.5, color="black",zorder=4)

    #拠点と台風の凡例表示
    r = patches.Rectangle(xy=(167, 50), width=20, height=8, ec='k', fc='w',zorder=3)
    ax.add_patch(r)
    
    ax.plot(view_lon,view_lat_1 - 0.8*d_view_lat,'crimson',markersize=15,marker="d",zorder=4)
    ax.text(text_lon, view_lat_1 - 0.8*d_view_lat-0.5, "Base", size=15, color="black",zorder=4)

    ax.plot(view_lon,view_lat_1 - 2*d_view_lat,'grey',markersize=30,marker="o",zorder=4)
    ax.text(text_lon, view_lat_1 - 2*d_view_lat-0.5, "Typhoon", size=15, color="black",zorder=4)



    #船の作図
    ship_lat = ship_typhoon_route_data[j,"TPGSHIP LAT"]
    ship_lon = ship_typhoon_route_data[j,"TPGSHIP LON"]
    if j == 0:
        u = 0
        v = 1
    else:
        ship_lat_2 = ship_typhoon_route_data[j-1,"TPGSHIP LAT"]
        ship_lon_2 = ship_typhoon_route_data[j-1,"TPGSHIP LON"]
        u = (ship_lon - ship_lon_2)
        v = (ship_lat - ship_lat_2)
        if (u == 0) & (v == 0):
            u = 0
            v = 1


    size = 10
    
    vec_size = (np.sqrt(u**2 + v**2))
    ax.quiver(ship_lon, ship_lat, u/vec_size*size, v/vec_size*size, color=iro, edgecolor="k", linewidth = 0.25, headlength=6 , headaxislength=6, width=.004, scale=400.,zorder=5)
    

    #j = j_ori

    plt.savefig("png_Folder_ver4/draw" + str(j) + ".png")

    im = Image.open("png_Folder_ver4/draw" + str(j) + ".png")

    im_crop = im.crop((110, 180, 780, 980))
    im_crop.save("png_Folder_ver4/draw" + str(j) + ".png", quality=100)


In [3]:
print(len(ship_typhoon_route_data))

1461


動画化

In [3]:
import cv2
import sys
# encoder(for mp4)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
# output file name, encoder, fps, size(fit to image size)
video = cv2.VideoWriter(str(year) + '_map_data_ver4_32.mp4',fourcc, 32.0, (670, 800))#サイズ揃える

if not video.isOpened():
    print("can't be opened")
    sys.exit()

for i in range(len(ship_typhoon_route_data)):
    img = cv2.imread('png_Folder_ver4/draw'+str(i)+'.png')


    # can't read image, escape
    if img is None:
        print("can't read")
        break
    

    # add
    video.write(img)

video.release()
print('written')

written
